In [1]:
num = 1

In [2]:
import os,shutil
import torchvision.transforms as trns
from PIL import Image
from scipy.io import loadmat
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import glob
import torch
import matplotlib.pyplot as plt
import numpy as np

In [3]:
print(torch.__version__)


1.6.0


In [4]:
# sml_Img = glob.glob("..\\pics\\sigrates\\small\\*")
# for i in sml_Img:
#     img = Image.open(i)
#     w, h = img.size
#     print(i, w,h)

In [12]:
## 不同解析度照片分開處理 Ricap

big_Img = glob.glob("C:\\Users\\user\\Desktop\\Class\\Cigarette_Brand\\Picture\\Cargo_Pic\\20200827_summarize\\Original03+05\\Pic\\1231_Ori+YUV+Rot\\Train_pics\\Ricap\\big\\*")
sml_Img = glob.glob("C:\\Users\\user\\Desktop\\Class\\Cigarette_Brand\\Picture\\Cargo_Pic\\20200827_summarize\\Original03+05\\Pic\\1231_Ori+YUV+Rot\\Train_pics\\Ricap\\small\\*")
len_data = len(big_Img) ## 若要執行小畫素照片 ()內要更改
print(len_data)
#
img_path = big_Img[:int(len_data)]
# img_path = sml_Img[:int(len_data)]
print(img_path)

52
['C:\\Users\\user\\Desktop\\Class\\Cigarette_Brand\\Picture\\Cargo_Pic\\20200827_summarize\\Original03+05\\Pic\\1231_Ori+YUV+Rot\\Train_pics\\Ricap\\big\\DSC_0088.JPG', 'C:\\Users\\user\\Desktop\\Class\\Cigarette_Brand\\Picture\\Cargo_Pic\\20200827_summarize\\Original03+05\\Pic\\1231_Ori+YUV+Rot\\Train_pics\\Ricap\\big\\DSC_0090.JPG', 'C:\\Users\\user\\Desktop\\Class\\Cigarette_Brand\\Picture\\Cargo_Pic\\20200827_summarize\\Original03+05\\Pic\\1231_Ori+YUV+Rot\\Train_pics\\Ricap\\big\\DSC_0091.JPG', 'C:\\Users\\user\\Desktop\\Class\\Cigarette_Brand\\Picture\\Cargo_Pic\\20200827_summarize\\Original03+05\\Pic\\1231_Ori+YUV+Rot\\Train_pics\\Ricap\\big\\DSC_0132.JPG', 'C:\\Users\\user\\Desktop\\Class\\Cigarette_Brand\\Picture\\Cargo_Pic\\20200827_summarize\\Original03+05\\Pic\\1231_Ori+YUV+Rot\\Train_pics\\Ricap\\big\\DSC_0133.JPG', 'C:\\Users\\user\\Desktop\\Class\\Cigarette_Brand\\Picture\\Cargo_Pic\\20200827_summarize\\Original03+05\\Pic\\1231_Ori+YUV+Rot\\Train_pics\\Ricap\\big\\DSC

In [13]:
class sigrateDataset(Dataset):
    def __init__(self, root, split, transform):
        # Initialize paths, transforms, and so on
        # --------------------------------------------
        self.imgs = img_path
        self.transform = transform

    def __getitem__(self, index):

        imgpath = self.imgs[index]
        img = Image.open(imgpath).convert('RGB')

        if self.transform is not None:
            img = self.transform(img)
        #return img, lbl
        return img

    def __len__(self):
        # --------------------------------------------
        # Indicate the total size of the dataset
        # --------------------------------------------
        return len(self.imgs)

In [14]:
train_transform = trns.Compose([
#     trns.Resize((1280, 960)),    ## small size pics 需resize到一樣
    #trns.RandomCrop((224, 224)),
    #trns.RandomHorizontalFlip(),
    trns.ToTensor(),
    #trns.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [15]:
## root 目錄要更改 big or small
path = "\\big"
train_set = sigrateDataset(root= "C:\\Users\\user\\Desktop\\Class\\Cigarette_Brand\\Picture\\Cargo_Pic\\20200827_summarize\\Original03+05\\Pic\\1231_Ori+YUV+Rot\\Train_pics\\Ricap"+path,
                           split='train', transform=train_transform)

In [16]:
train_loader = torch.utils.data.DataLoader(
        train_set,
        batch_size=6,
        num_workers=0,
        shuffle=True
    )

In [17]:
# for i,data in enumerate(train_loader):
#     plt.imshow(np.transpose(data[1].cpu().detach().numpy(), (1, 2, 0)))

In [18]:
# for imgs in train_loader:
#     plt.imshow(np.transpose(imgs[0].cpu().detach().numpy(), (1, 2, 0)))
#     print('Size of image:', imgs.size()) 
#     print('Type of image:', imgs.dtype)

In [19]:
def ricap(loaders, beta=0.3):
    
    ## Set boundary position's beta parameter (beta, beta)
    ## Calculate w,h
    flag1 = True
    flag2 = True
    while flag1:
        w_ = round(float(np.random.beta(beta,beta)),3)
        if w_ > 0.15 and w_ < 0.35:
            w_ = w_
            print("w'= ", w_)
            flag1 = False
        elif w_ > 0.7 and w_ < 0.9:
            w_ = w_
            print("w'= ", w_)
            flag1 = False    
    while flag2:
        h_ = round(float(np.random.beta(beta,beta)),3)
        if h_ > 0.15 and h_ < 0.35:
            h_ = h_
            print("h'= ", h_)
            flag2 = False
        elif h_ > 0.7 and h_ < 0.9:
            h_ = h_
            print("h'= ", h_)
            flag2 = False
    
    
    for target, inputs in enumerate(loaders):
        I_r, I_c = inputs.size()[2:]
        print("I_r: ", I_r)
        print("I_c: ", I_c)
        w = int(np.round(I_r * h_))
        h = int(np.round(I_c * w_))
        w_dash = [w, I_r - w, w, I_r - w]
        h_dash = [h, h, I_c - h, I_c - h]
#         print(w_dash)
#         print(h_dash)
        cropped_images = {}
        c_ = {}
        W_ = {}
        for k in range(4):
                idx = torch.randperm(inputs.size(0))
                x_k = np.random.randint(0, I_r - w_dash[k] + 1)
                y_k = np.random.randint(0, I_c - h_dash[k] + 1)
                cropped_images[k] = inputs[idx][:, :, x_k:x_k + w_dash[k], y_k:y_k + h_dash[k]]
#                 c_[k] = target[idx].cuda()
                W_[k] = w_dash[k] * h_dash[k] / (I_r * I_c)
                
        patched_images = torch.cat(
                (torch.cat((cropped_images[0], cropped_images[1]), 2),
                torch.cat((cropped_images[2], cropped_images[3]), 2)),
            3)
        
        patched_images = patched_images.cuda()
        return patched_images

In [20]:
for i in range(17):
    ricap_imgs = ricap(train_loader)
    print(ricap_imgs.size())
    print(ricap_imgs.dtype)
    result_path = "C:\\Users\\user\\Desktop\\Class\\Cigarette_Brand\\Picture\\Cargo_Pic\\20200827_summarize\\Original03+05\\Pic\\1231_Ori+YUV+Rot\\Train_pics\\Ricap\\ricap_results\\"    
    for i in range(len(ricap_imgs[:])):
        img = np.transpose(ricap_imgs[i].cpu().detach().numpy(), (1,2,0))
        plt.imsave(result_path+"Ricap"+str(num)+".jpg", img)
        num+=1
    #     plt.imshow(img)
    

w'=  0.888
h'=  0.74
I_r:  3264
I_c:  4928
torch.Size([6, 3, 3264, 4928])
torch.float32
w'=  0.217
h'=  0.314
I_r:  3264
I_c:  4928
torch.Size([6, 3, 3264, 4928])
torch.float32
w'=  0.839
h'=  0.292
I_r:  3264
I_c:  4928
torch.Size([6, 3, 3264, 4928])
torch.float32
w'=  0.791
h'=  0.246
I_r:  3264
I_c:  4928
torch.Size([6, 3, 3264, 4928])
torch.float32
w'=  0.839
h'=  0.277
I_r:  3264
I_c:  4928
torch.Size([6, 3, 3264, 4928])
torch.float32
w'=  0.177
h'=  0.254
I_r:  3264
I_c:  4928
torch.Size([6, 3, 3264, 4928])
torch.float32
w'=  0.858
h'=  0.206
I_r:  3264
I_c:  4928
torch.Size([6, 3, 3264, 4928])
torch.float32
w'=  0.211
h'=  0.228
I_r:  3264
I_c:  4928
torch.Size([6, 3, 3264, 4928])
torch.float32
w'=  0.702
h'=  0.767
I_r:  3264
I_c:  4928
torch.Size([6, 3, 3264, 4928])
torch.float32
w'=  0.155
h'=  0.257
I_r:  3264
I_c:  4928
torch.Size([6, 3, 3264, 4928])
torch.float32
w'=  0.262
h'=  0.255
I_r:  3264
I_c:  4928
torch.Size([6, 3, 3264, 4928])
torch.float32
w'=  0.898
h'=  0.323
